In [ ]:
import matplotlib.pyplot as plt
import logging, os, sys, time
import tensorflow as tf
from model import file_input_fn, lm_model_fn
from utils import data_utils_py3
from os.path import expanduser
HOME = expanduser("~")

In [ ]:
def lr_range_test(
    lm_model_fn,
    train_input_fn,
    params):
    """
    train the model and evaluate every eval_every steps
    """
    
#     gpu_id = '2'
#     session_config = tf.ConfigProto(allow_soft_placement=True, gpu_options=tf.GPUOptions(visible_device_list=gpu_id))
#     strategy = tf.distribute.MirroredStrategy()
    config=tf.estimator.RunConfig(
#         train_distribute=strategy,
#         eval_distribute=strategy,
#         session_config=session_config,
        save_checkpoints_steps=None,
        save_checkpoints_secs=None,
        log_step_count_steps=10)
    local_params = {}
    local_params.update(params)
    local_params['schedule'] = 'lr_finder'
    lm = tf.estimator.Estimator(
        model_fn=lm_model_fn,
        model_dir='',
        params=local_params,
        config=config)
    
    # start lr range test
    lm.train(
        input_fn=train_input_fn,
        steps=1000)

In [ ]:
def train_and_evaluate(
    lm_model_fn,
    train_input_fn,
    eval_input_fn,
    train_dir,
    params,
    eval_every=10000):
    """
    train the model and evaluate every eval_every steps
    """
    
#     gpu_id = '2'
#     session_config = tf.ConfigProto(allow_soft_placement=True, gpu_options=tf.GPUOptions(visible_device_list=gpu_id))
    strategy = tf.distribute.MirroredStrategy()
    config=tf.estimator.RunConfig(
        train_distribute=strategy,
        eval_distribute=strategy,
#         session_config=session_config,
        log_step_count_steps=1000)
    local_params = {}
    local_params.update(params)
    local_params['schedule'] = '1cycle'
    lm = tf.estimator.Estimator(
        model_fn=lm_model_fn,
        model_dir=train_dir,
        params=local_params,
        config=config)
    
    # get TF logger
    log = logging.getLogger('tensorflow')
    log.setLevel(logging.DEBUG)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    # create file handler which logs even debug messages
    os.makedirs(train_dir, exist_ok=True)
    fh = logging.FileHandler(os.path.join(train_dir, 'tensorflow.log'))
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(formatter)
    log.addHandler(fh)
    
    # start train and eval loop
    for _ in range(int(params['num_steps'] / eval_every)):
        lm.train(
            input_fn=train_input_fn,
            steps=eval_every)
        lm.evaluate(
            input_fn=eval_input_fn)

In [ ]:
char_vocab_file = HOME+"/Data/Vocab/vocab_zh"
word_vocab_file = HOME+"/Data/Vocab/word_vocab_zh"
char_embedding_files = HOME+"/Data/Vocab/zh_char_300_nlpcc.txt"
data_dir = HOME+"/Data/text_zh"
train_dir = 'estimator_zh_small'

char_vocab = data_utils_py3.Vocab(
    char_vocab_file,
    embedding_files=char_embedding_files)
char_vocab_size = char_vocab.size()

params = {
    'max_lr': 5e-5,
    'num_steps': 200000,
    'pct_start': 0.3,
    'dropout': 0.1,
    'wd': .0,
    'char_vocab_size': char_vocab_size,
    'char_vocab_dim': 300,
    'char_vocab_emb': char_vocab.embedding_init,
    'max_char_length': 8,
    'layer_size': 256,
    'num_layers': 3,
    'data_spec': [{'type': 'seq', 'is_target': True, 'select_from': None}]
}


batch_size = 32
num_pieces = 1
max_lengths = [256]
train_input_fn = lambda: file_input_fn(
    char_vocab, os.path.join(data_dir, 'train'),
    num_pieces, batch_size, max_lengths, True)
eval_input_fn = lambda: file_input_fn(
    char_vocab, os.path.join(data_dir, 'dev'),
    num_pieces, batch_size, max_lengths, False)

In [ ]:
lr_range_test(lm_model_fn, train_input_fn, params)

In [ ]:
train_and_evaluate(
    lm_model_fn, train_input_fn, eval_input_fn, train_dir, params, eval_every=10000)